In [2]:
import numpy as np
import pandas as pd

from lib.script_scraper import ScriptScraper
from lib.corpus_enhancer import CorpusEnhancer

In [2]:
series_url = 'https://subslikescript.com/series/Parks_and_Recreation-1266020'
series_id = 'parksandrec'
acts = 3
scenes = 5
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'

In [3]:
pnr_scraper = ScriptScraper(
    series_url = series_url,
    series_id = series_id,
    user_agent = user_agent,
    acts = acts,
    scenes = scenes
)

In [4]:
pnr_corpus = pnr_scraper.generate_corpus(season_limit = 1)

In [5]:
pnr_corpus

line
series_id   season_id episode_id episode_title scene_id line_id                                         
parksandrec 1         1          Pilot         1        1                                     Hello. Hi.
                                                        2        My name is Leslie Knope, and I work for
                                                        3           the Parks and Recreation Department.
                                                        4                 Can I ask you a few questions?
                                                        5                    Would you say that you are,
...                                                                                                  ...
                      6          Rock Show     15       60                   We all were in it, the pit!
                                                        61                                      The pit!
                                                        62                         I was in it, the pit!
                                                        63                       You fell in it, the pit
                                                        64                       We all fell in that pit

[4861 rows x 1 columns]

In [6]:
pnr_tokens = pnr_scraper.generate_tokens()

In [7]:
pnr_tokens

token_str  \
series_id   season_id episode_id episode_title scene_id line_id token_id             
parksandrec 1         1          Pilot         1        1       0           Hello.   
                                                                1              Hi.   
                                                        2       0               My   
                                                                1             name   
                                                                2               is   
...                                                                            ...   
                      6          Rock Show     15       64      1              all   
                                                                2             fell   
                                                                3               in   
                                                                4             that   
                                                                5              pit   

                                                                         term_str  
series_id   season_id episode_id episode_title scene_id line_id token_id           
parksandrec 1         1          Pilot         1        1       0           hello  
                                                                1              hi  
                                                        2       0              my  
                                                                1            name  
                                                                2              is  
...                                                                           ...  
                      6          Rock Show     15       64      1             all  
                                                                2            fell  
                                                                3              in  
                                                                4            that  
                                                                5             pit  

[20228 rows x 2 columns]

In [8]:
pnr_vocab = pnr_scraper.generate_vocab()

In [9]:
pnr_vocab

,n
term_str,
i,712
you,668
the,545
a,535
to,450
...,...
gambit,1
topics,1
smear,1


In [10]:
pnr_corpus.to_csv('PNR-CORPUS.csv')
pnr_tokens.to_csv('PNR-TOKENS.csv')
pnr_vocab.to_csv('PNR-VOCAB.csv')

In [8]:
OHCO = ['series_id', 'season_id', 'episode_id', 'scene_id', 'line_id', 'token_id']
TOKENS = OHCO[:6]
LINES = OHCO[:5]
SCENES = OHCO[:4]
EPISODES = OHCO[:3]
SEASONS = OHCO[:2]
SERIES = OHCO[:1]

In [9]:
pnr_tokens = pd.read_csv('PNR-TOKENS.csv')

In [10]:
pnr_tokens = pnr_tokens.drop('episode_title', axis = 1).set_index(TOKENS)

In [11]:
pnr_tokens

token_str term_str
series_id   season_id episode_id scene_id line_id token_id                   
parksandrec 1         1          1        1       0           Hello.    hello
                                                  1              Hi.       hi
                                          2       0               My       my
                                                  1             name     name
                                                  2               is       is
...                                                              ...      ...
                      6          15       64      1              all      all
                                                  2             fell     fell
                                                  3               in       in
                                                  4             that     that
                                                  5              pit      pit

[20228 rows x 2 columns]

In [16]:
pnr_vocab = pd.read_csv('PNR-VOCAB.csv').set_index('term_str')
pnr_vocab

,n
term_str,
i,712
you,668
the,545
a,535
to,450
...,...
gambit,1
topics,1
smear,1


In [17]:
pnr_corp_enhancer = CorpusEnhancer(corpus = pnr_tokens, vocab = pnr_vocab)

In [20]:
pnr_vocab_enhanced = pnr_corp_enhancer.enhance_vocab()

In [21]:
pnr_corp_enhanced = pnr_corp_enhancer.enhance_corpus(strip_hyphens = True, strip_whitespace = True)

In [24]:
pnr_corp_enhanced[pnr_corp_enhanced.pos == 'NNP']

pos_tuple  \
series_id   season_id episode_id scene_id line_id token_id                      
parksandrec 1         1          1        2       3        0 0  (Leslie, NNP)   
                                                  4        0 0  (Knope,, NNP)   
                                          15      0        0 0     (Ms., NNP)   
                                                  1        0 0  (Knope,, NNP)   
                                 2        14      1        0 0  (Leslie, NNP)   
...                                                                       ...   
                      6          13       54      1        0 0  (Harris, NNP)   
                                 14       2       2        0 0     (Dr., NNP)   
                                          4       0        0 0     (Dr., NNP)   
                                                  1        0 0  (Harris, NNP)   
                                          7       7        0 0     (Dr., NNP)   

                                                                pos token_str  \
series_id   season_id episode_id scene_id line_id token_id                      
parksandrec 1         1          1        2       3        0 0  NNP    Leslie   
                                                  4        0 0  NNP    Knope,   
                                          15      0        0 0  NNP       Ms.   
                                                  1        0 0  NNP    Knope,   
                                 2        14      1        0 0  NNP    Leslie   
...                                                             ...       ...   
                      6          13       54      1        0 0  NNP    Harris   
                                 14       2       2        0 0  NNP       Dr.   
                                          4       0        0 0  NNP       Dr.   
                                                  1        0 0  NNP    Harris   
                                          7       7        0 0  NNP       Dr.   

                                                               term_str  
series_id   season_id episode_id scene_id line_id token_id               
parksandrec 1         1          1        2       3        0 0   leslie  
                                                  4        0 0   knope,  
                                          15      0        0 0      ms.  
                                                  1        0 0   knope,  
                                 2        14      1        0 0   leslie  
...                                                                 ...  
                      6          13       54      1        0 0   harris  
                                 14       2       2        0 0      dr.  
                                          4       0        0 0      dr.  
                                                  1        0 0   harris  
                                          7       7        0 0      dr.  

[230 rows x 4 columns]